# Predict heart disease project

In [1]:
# predict heart disease project

In [22]:
#libraries
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import joblib
import warnings

warnings.filterwarnings("ignore")

In [3]:
#read data
df=pd.read_csv("/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv")

In [4]:
#EDA
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
df.shape

(4238, 16)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   int64  
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   float64
 6   prevalentStroke  4238 non-null   int64  
 7   prevalentHyp     4238 non-null   int64  
 8   diabetes         4238 non-null   int64  
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  TenYearCHD       4238 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 529.9 KB


In [9]:
df.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [10]:
def auto_fill_missing_values(df, exclude_columns=None):
    """
    Automatically fill missing values in a DataFrame, excluding specified columns.
    """
    # Separate columns by data type
    numerical_columns = df.select_dtypes(include='number').columns
    categorical_columns = df.select_dtypes(exclude='number').columns

    # Exclude specified columns
    if exclude_columns:
        numerical_columns = numerical_columns.difference(exclude_columns)
        categorical_columns = categorical_columns.difference(exclude_columns)

    # Fill missing values for numerical columns with the mean
    df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())

    # Fill missing values for categorical columns with the mode
    df[categorical_columns] = df[categorical_columns].apply(lambda x: x.fillna(x.mode().iloc[0]))

    return df

# Example usage: Exclude columns 'column1' and 'column2'
df = auto_fill_missing_values(df)
#belong to chatgpt

In [11]:
df.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [13]:
df.corr()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
male,1.000000,-0.028979,0.017126,0.197596,0.316807,-0.052204,-0.004546,0.005313,0.015708,-0.069974,-0.035989,0.057933,0.081506,-0.116601,0.005818,0.088428
age,-0.028979,1.000000,-0.163613,-0.213748,-0.192366,0.121980,0.057655,0.307194,0.101258,0.260270,0.394302,0.206104,0.135283,-0.012819,0.116850,0.225256
education,0.017126,-0.163613,1.000000,0.018301,0.007962,-0.010607,-0.035110,-0.080993,-0.038146,-0.022507,-0.128260,-0.061755,-0.135635,-0.053626,-0.033721,-0.053384
currentSmoker,0.197596,-0.213748,0.018301,1.000000,0.766970,-0.048632,-0.032988,-0.103260,-0.044295,-0.046285,-0.130230,-0.107746,-0.167276,0.062348,-0.054157,0.019456
cigsPerDay,0.316807,-0.192366,0.007962,0.766970,1.000000,-0.045826,-0.032706,-0.065947,-0.037063,-0.026025,-0.088505,-0.056391,-0.092453,0.074851,-0.056088,0.057775
BPMeds,-0.052204,0.121980,-0.010607,-0.048632,-0.045826,1.000000,0.115003,0.259243,0.051571,0.078909,0.252047,0.192490,0.099552,0.015175,0.048905,0.086774
prevalentStroke,-0.004546,0.057655,-0.035110,-0.032988,-0.032706,0.115003,1.000000,0.074830,0.006949,0.000067,0.057009,0.045190,0.024840,-0.017676,0.018055,0.061810
prevalentHyp,0.005313,0.307194,-0.080993,-0.103260,-0.065947,0.259243,0.074830,1.000000,0.077808,0.163041,0.696755,0.615751,0.300572,0.147222,0.082924,0.177603
diabetes,0.015708,0.101258,-0.038146,-0.044295,-0.037063,0.051571,0.006949,0.077808,1.000000,0.040092,0.111283,0.050329,0.086250,0.048993,0.605705,0.097317
totChol,-0.069974,0.260270,-0.022507,-0.046285,-0.026025,0.078909,0.000067,0.163041,0.040092,1.000000,0.207609,0.163903,0.114789,0.090676,0.044583,0.081624


In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
male,4238.0,0.429212,0.495022,0.00,0.00,0.00,1.0000,1.0
age,4238.0,49.584946,8.572160,32.00,42.00,49.00,56.0000,70.0
education,4238.0,1.978950,1.007075,1.00,1.00,2.00,3.0000,4.0
currentSmoker,4238.0,0.494101,0.500024,0.00,0.00,0.00,1.0000,1.0
cigsPerDay,4238.0,9.003089,11.879230,0.00,0.00,0.00,20.0000,70.0
BPMeds,4238.0,0.029630,0.168520,0.00,0.00,0.00,0.0000,1.0
prevalentStroke,4238.0,0.005899,0.076587,0.00,0.00,0.00,0.0000,1.0
prevalentHyp,4238.0,0.310524,0.462763,0.00,0.00,0.00,1.0000,1.0
diabetes,4238.0,0.025720,0.158316,0.00,0.00,0.00,0.0000,1.0
totChol,4238.0,236.721585,44.326453,107.00,206.00,234.00,262.0000,696.0


In [16]:
df["TenYearCHD"].unique()

array([0, 1])

In [17]:
#preprocesing

In [24]:
# x y split
X=df.drop("TenYearCHD",axis=1)
y=df["TenYearCHD"]

In [25]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [30]:
#chek data
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (3390, 15)
X_test shape: (848, 15)
y_train shape: (3390,)
y_test shape: (848,)


In [29]:
#  the neural network model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=15))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [31]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/30
106/106 [==============================] - 1s 4ms/step - loss: 0.4412 - accuracy: 0.8475 - val_loss: 0.3903 - val_accuracy: 0.8526
Epoch 2/30
106/106 [==============================] - 0s 2ms/step - loss: 0.3914 - accuracy: 0.8487 - val_loss: 0.3916 - val_accuracy: 0.8514
Epoch 3/30
106/106 [==============================] - 0s 3ms/step - loss: 0.3835 - accuracy: 0.8525 - val_loss: 0.3921 - val_accuracy: 0.8550
Epoch 4/30
106/106 [==============================] - 0s 3ms/step - loss: 0.3794 - accuracy: 0.8534 - val_loss: 0.3905 - val_accuracy: 0.8573
Epoch 5/30
106/106 [==============================] - 0s 2ms/step - loss: 0.3740 - accuracy: 0.8555 - val_loss: 0.3847 - val_accuracy: 0.8561
Epoch 6/30
106/106 [==============================] - 0s 3ms/step - loss: 0.3716 - accuracy: 0.8540 - val_loss: 0.3842 - val_accuracy: 0.8561
Epoch 7/30
106/106 [==============================] - 0s 2ms/step - loss: 0.3665 - accuracy: 0.8558 - val_loss: 0.3832 - val_accuracy: 0.8597
Epoch 

In [37]:
accuracy = model.evaluate(X_test, y_test)[1]
print("Validation data accruacy:", accuracy)

27/27 [==============================] - 0s 1ms/step - loss: 0.4293 - accuracy: 0.8337
Validation data accruacy: 0.8337264060974121


In [39]:
#save model
joblib.dump(model,"model.joblib")

['model.joblib']

-Zafer Acar Notes,-[https://chat.openai.com/]